In [1]:
import nltk

In [2]:
# movie review sentences
from nltk.corpus import sentence_polarity
import random

In [3]:
# get the sentence corpus and look at some sentences
sentences = sentence_polarity.sents()
print(len(sentences))
print(type(sentences))
print(sentence_polarity.categories())
# sentences are already tokenized, print the first four sentences
for sent in sentences[:4]:
    print(sent)

10662
<class 'nltk.corpus.reader.util.ConcatenatedCorpusView'>
['neg', 'pos']
['simplistic', ',', 'silly', 'and', 'tedious', '.']
["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.']
['exploitative', 'and', 'largely', 'devoid', 'of', 'the', 'depth', 'or', 'sophistication', 'that', 'would', 'make', 'watching', 'such', 'a', 'graphic', 'treatment', 'of', 'the', 'crimes', 'bearable', '.']
['[garbus]', 'discards', 'the', 'potential', 'for', 'pathological', 'study', ',', 'exhuming', 'instead', ',', 'the', 'skewed', 'melodrama', 'of', 'the', 'circumstantial', 'situation', '.']


In [4]:
# look at the sentences by category to see how many positive and negative
pos_sents = sentence_polarity.sents(categories='pos')
print(len(pos_sents))
neg_sents = sentence_polarity.sents(categories='neg')
print(len(neg_sents))

5331
5331


In [5]:
## setup the movie reviews sentences for classification
# create a list of documents, each document is one sentence as a list of words paired with category
documents = [(sent, cat) for cat in sentence_polarity.categories() 
	for sent in sentence_polarity.sents(categories=cat)]

In [6]:
# look at the first and last documents - consists of all the words in the review
# followed by the category
print(documents[0])
print(documents[-1])
# randomly reorder documents
random.shuffle(documents)

(['simplistic', ',', 'silly', 'and', 'tedious', '.'], 'neg')
(['provides', 'a', 'porthole', 'into', 'that', 'noble', ',', 'trembling', 'incoherence', 'that', 'defines', 'us', 'all', '.'], 'pos')


In [7]:
# get all words from all movie_reviews and put into a frequency distribution
#   note lowercase, but no stemming or stopwords
all_words_list = [word for (sent,cat) in documents for word in sent]
all_words = nltk.FreqDist(all_words_list)
# get the 2000 most frequently appearing keywords in the corpus
word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]
print(word_features[:50])

['.', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'that', 'it', 'as', 'but', 'with', 'film', 'this', 'for', 'its', 'an', 'movie', "it's", 'be', 'on', 'you', 'not', 'by', 'about', 'one', 'more', 'like', 'has', 'are', 'at', 'from', 'than', '"', 'all', '--', 'his', 'have', 'so', 'if', 'or', 'story', 'i', 'too', 'just', 'who', 'into', 'what']


In [8]:
# define features (keywords) of a document for a BOW/unigram baseline
# each feature is 'contains(keyword)' and is true or false depending
# on whether that keyword is in the document
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    return features

In [9]:
# get features sets for a document, including keyword features and category feature
featuresets = [(document_features(d, word_features), c) for (d, c) in documents]

# the feature sets are 2000 words long so you may not want to look at one
featuresets[0]

({'V_.': True,
  'V_the': True,
  'V_,': False,
  'V_a': False,
  'V_and': False,
  'V_of': True,
  'V_to': False,
  'V_is': True,
  'V_in': False,
  'V_that': False,
  'V_it': False,
  'V_as': False,
  'V_but': False,
  'V_with': False,
  'V_film': True,
  'V_this': True,
  'V_for': False,
  'V_its': False,
  'V_an': True,
  'V_movie': False,
  "V_it's": False,
  'V_be': False,
  'V_on': True,
  'V_you': False,
  'V_not': False,
  'V_by': False,
  'V_about': False,
  'V_one': False,
  'V_more': False,
  'V_like': False,
  'V_has': False,
  'V_are': False,
  'V_at': False,
  'V_from': True,
  'V_than': False,
  'V_"': False,
  'V_all': False,
  'V_--': False,
  'V_his': True,
  'V_have': False,
  'V_so': False,
  'V_if': False,
  'V_or': False,
  'V_story': False,
  'V_i': False,
  'V_too': False,
  'V_just': False,
  'V_who': True,
  'V_into': False,
  'V_what': False,
  'V_most': False,
  'V_out': False,
  'V_no': False,
  'V_much': False,
  'V_even': False,
  'V_good': False,
  'V_u

In [10]:
# training using naive Baysian classifier, training set is approximately 90% of data
train_set, test_set = featuresets[1000:], featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [11]:
# evaluate the accuracy of the classifier
nltk.classify.accuracy(classifier, test_set)

0.758

In [12]:
# show which features of classifier are most informative
classifier.show_most_informative_features(30)

Most Informative Features
            V_engrossing = True              pos : neg    =     20.4 : 1.0
              V_captures = True              pos : neg    =     18.4 : 1.0
               V_generic = True              neg : pos    =     17.0 : 1.0
              V_mediocre = True              neg : pos    =     17.0 : 1.0
             V_inventive = True              pos : neg    =     15.0 : 1.0
                  V_flat = True              neg : pos    =     14.6 : 1.0
                V_unique = True              pos : neg    =     14.4 : 1.0
                V_boring = True              neg : pos    =     13.8 : 1.0
                  V_warm = True              pos : neg    =     13.0 : 1.0
             V_wonderful = True              pos : neg    =     13.0 : 1.0
               V_routine = True              neg : pos    =     13.0 : 1.0
             V_absorbing = True              pos : neg    =     12.3 : 1.0
            V_refreshing = True              pos : neg    =     12.3 : 1.0

In [14]:
import Subjectivity

In [15]:
# create your own path to the subjclues file
SLpath = "subjclueslen1-HLTEMNLP05/subjclueslen1-HLTEMNLP05.tff"

In [16]:
SL = Subjectivity.readSubjectivity(SLpath)

In [17]:
# how many words are in the dictionary
len(SL.keys())

6885

In [18]:
# look at words in the dictionary
print(SL['absolute'])
print(SL['shabby'])
# note what happens if the word is not there
print(SL['dog'])

['strongsubj', 'adj', False, 'neutral']
['strongsubj', 'adj', False, 'negative']


KeyError: 'dog'

In [19]:
# use multiple assignment to get the 4 items
strength, posTag, isStemmed, polarity = SL['absolute']
print(polarity)

neutral


In [20]:
# define features that include word counts of subjectivity words
# negative feature will have number of weakly negative words +
#    2 * number of strongly negative words
# positive feature has similar definition
#    not counting neutral words
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    for word in document_words:
        if word in SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)      
    return features

In [21]:
SL_featuresets = [(SL_features(d, word_features, SL), c) for (d, c) in documents]

In [27]:
# show document 0 and just the two sentiment lexicon features 
print(documents[0])
print(SL_featuresets[0][0]['positivecount'])
print(SL_featuresets[0][0]['negativecount'])

(['this', 'is', 'an', 'undeniably', 'intriguing', 'film', 'from', 'an', 'adventurous', 'young', 'talent', 'who', 'finds', 'his', 'inspiration', 'on', 'the', 'fringes', 'of', 'the', 'american', 'underground', '.'], 'pos')
7
0


In [25]:
# this gives the label of document 0
print(SL_featuresets[0][1])
# number of features for document 0
print(len(SL_featuresets[0][0].keys()))

pos
2002


In [26]:
# retrain the classifier using these features
train_set, test_set = SL_featuresets[1000:], SL_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.774

In [28]:
for sent in list(sentences)[:50]:
   for word in sent:
     if (word.endswith("n't")):
       print(sent)

['there', 'is', 'a', 'difference', 'between', 'movies', 'with', 'the', 'courage', 'to', 'go', 'over', 'the', 'top', 'and', 'movies', 'that', "don't", 'care', 'about', 'being', 'stupid']
['a', 'farce', 'of', 'a', 'parody', 'of', 'a', 'comedy', 'of', 'a', 'premise', ',', 'it', "isn't", 'a', 'comparison', 'to', 'reality', 'so', 'much', 'as', 'it', 'is', 'a', 'commentary', 'about', 'our', 'knowledge', 'of', 'films', '.']
['i', "didn't", 'laugh', '.', 'i', "didn't", 'smile', '.', 'i', 'survived', '.']
['i', "didn't", 'laugh', '.', 'i', "didn't", 'smile', '.', 'i', 'survived', '.']
['most', 'of', 'the', 'problems', 'with', 'the', 'film', "don't", 'derive', 'from', 'the', 'screenplay', ',', 'but', 'rather', 'the', 'mediocre', 'performances', 'by', 'most', 'of', 'the', 'actors', 'involved']
['the', 'lack', 'of', 'naturalness', 'makes', 'everything', 'seem', 'self-consciously', 'poetic', 'and', 'forced', '.', '.', '.', "it's", 'a', 'pity', 'that', "[nelson's]", 'achievement', "doesn't", 'match'

In [29]:
# this list of negation words includes some "approximate negators" like hardly and rarely
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

In [30]:
# One strategy with negation words is to negate the word following the negation word
#   other strategies negate all words up to the next punctuation
# Strategy is to go through the document words in order adding the word features,
#   but if the word follows a negation words, change the feature to negated word
# Start the feature set with all 2000 word features and 2000 Not word features set to false
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = False
        features['V_NOT{}'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['V_NOT{}'.format(document[i])] = (document[i] in word_features)
        else:
            features['V_{}'.format(word)] = (word in word_features)
    return features

In [31]:
# define the feature sets
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in documents]
# show the values of a couple of example features
print(NOT_featuresets[0][0]['V_NOTcare'])
print(NOT_featuresets[0][0]['V_always'])

False
False


In [ ]:
train_set, test_set = NOT_featuresets[1000:], NOT_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)